In [1]:
import os
import pandas as pd
from pcp_module import process_audio_and_save_pcp
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
def process_annotation_file(annotation_file_path):
    """Process a single annotation file and return the processed DataFrame."""
    with open(annotation_file_path, 'r') as file:
        lines = file.readlines()[7:]    
        arff_content = [line.strip().strip("'").split(",") for line in lines]
    
    annotations_df = pd.DataFrame(arff_content, columns=['start_time', 'bar', 'beat', 'chord'])
    annotations_df['start_time'] = annotations_df['start_time'].astype(float)
    annotations_df['bar'] = annotations_df['bar'].astype(int)
    annotations_df['beat'] = annotations_df['beat'].astype(int)
    annotations_df['chord'] = annotations_df['chord'].str.strip("'")
    annotations_df['end_time'] = annotations_df['start_time'].shift(-1)
    annotations_df = annotations_df.ffill()
    
    return annotations_df

def process_file_pair(args):
    """Process a pair of audio and annotation files."""
    audio_file_name, annotations_file_name, dataset_location, annotations_dir_loc, output_dir = args
    try:
        # Process annotation file
        annotation_file_path = os.path.join(annotations_dir_loc, annotations_file_name)
        annotations_df = process_annotation_file(annotation_file_path)
        
        # Process audio and save PCP
        process_audio_and_save_pcp(audio_file_name, dataset_location, annotations_df, output_dir)
        
        return f"Successfully processed {audio_file_name}"
    except Exception as e:
        return f"Error processing {audio_file_name}: {str(e)}"

def main():
    # Configuration
    dataset_location = "./tinyaam/audio-mixes-mp3/"
    output_dir = 'pcpvectors'
    annotations_dir_loc = "./tinyaam/annotations/"
    
    # Get file lists
    audio_file_names = [file for root, dirs, files in os.walk(dataset_location) for file in files]
    arff_files = [file for root, dirs, files in os.walk(annotations_dir_loc) 
                  for file in files if file.endswith('beatinfo.arff')]
    
    # Prepare arguments for parallel processing
    process_args = [
        (audio_file, arff_file, dataset_location, annotations_dir_loc, output_dir)
        for audio_file, arff_file in zip(audio_file_names, arff_files)
    ]
    
    # Calculate optimal number of workers
    num_workers = min(len(process_args), os.cpu_count() * 2)  # 2 threads per CPU core
    
    # Process files in parallel with progress bar
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all tasks
        future_to_file = {
            executor.submit(process_file_pair, args): args[0]
            for args in process_args
        }
        
        # Create progress bar
        with tqdm(total=len(process_args), unit=' Files') as pbar:
            for future in as_completed(future_to_file):
                file_name = future_to_file[future]
                try:
                    result = future.result()
                    if "Error" in result:
                        print(f"\nWarning: {result}")
                except Exception as e:
                    print(f"\nError processing {file_name}: {str(e)}")
                finally:
                    pbar.update(1)

if __name__ == "__main__":
    main()

In [4]:
dataset_location = "./tinyaam/audio-mixes-mp3/"
output_dir = 'pcpvectorssingle'
annotations_dir_loc = "./tinyaam/annotations/"

audio_file_names = [file for root, dirs, files in os.walk(dataset_location) for file in files]
arff_files = [file for root, dirs, files in os.walk(annotations_dir_loc) for file in files if file.endswith('beatinfo.arff')]
for audio_file_name, annotations_file_name in tqdm(zip(audio_file_names, arff_files), unit=' Files'):
    with open(os.path.join(annotations_dir_loc,annotations_file_name), 'r') as file:
        lines = file.readlines()[7:]    
        arff_content = [line.strip().strip("'").split(",") for line in lines]
    annotations_df = pd.DataFrame(arff_content, columns=['start_time', 'bar', 'beat', 'chord'])
    annotations_df['start_time'] = annotations_df['start_time'].astype(float)
    annotations_df['bar'] = annotations_df['bar'].astype(int)
    annotations_df['beat'] = annotations_df['beat'].astype(int)
    annotations_df['chord'] = annotations_df['chord'].str.strip("'")
    annotations_df['end_time'] = annotations_df['start_time'].shift(-1)
    annotations_df = annotations_df.ffill()
    process_audio_and_save_pcp(audio_file_name, dataset_location, annotations_df, output_dir)



0 Files [00:00, ? Files/s]

Loading audio file: 0001_mix.mp3
Audio loaded with sampling rate: 44100 Hz
Samples per microsecond: 0.0441


183 segments [00:22,  8.14 segments/s]
1 Files [00:24, 24.35s/ Files]

Processed: 0001_mix.mp3
Loading audio file: 0080_mix.mp3
Audio loaded with sampling rate: 44100 Hz
Samples per microsecond: 0.0441


255 segments [00:18, 13.63 segments/s]
2 Files [00:43, 21.09s/ Files]

...

Processed: 2841_mix.mp3
Loading audio file: 2990_mix.mp3
Audio loaded with sampling rate: 44100 Hz
Samples per microsecond: 0.0441


367 segments [00:40,  9.00 segments/s]
19 Files [10:08, 37.12s/ Files]

Processed: 2990_mix.mp3
Loading audio file: 3000_mix.mp3
Audio loaded with sampling rate: 44100 Hz
Samples per microsecond: 0.0441


191 segments [00:23,  8.01 segments/s]
20 Files [10:31, 31.60s/ Files]

Processed: 3000_mix.mp3
